In [ ]:
'''
PURPOSE: 
Download images from web into a folder.
    should I store each actor in a different folder? 
    Do not download if images take too long
    -----Do not download if broken link
    -----save images as actorname_number
Crop to bounding boxes, convert to grayscale and resize images. 
'''

In [358]:
import urllib
import os
import numpy as np
import pandas as pd
from random import randrange
import sys
from scipy.misc import imread, imshow, imresize, imsave
import matplotlib.pyplot as plt

In [359]:
def timeout(func, args=(), kwargs={}, timeout_duration=1, default=None):
    '''From:
    http://code.activestate.com/recipes/473878-timeout-function-using-threading/'''
    import threading
    class InterruptableThread(threading.Thread):
        def __init__(self):
            threading.Thread.__init__(self)
            self.result = None

        def run(self):
            try:
                self.result = func(*args, **kwargs)
            except:
                self.result = default

    it = InterruptableThread()
    it.start()
    it.join(timeout_duration)
    if it.isAlive():
        return False
    else:
        return it.result

In [360]:
def rgb2gray(rgb):
    '''Return the grayscale version of the RGB image rgb as a 2D numpy array
    whose range is 0..1
    Arguments:
    rgb -- an RGB image, represented as a numpy array of size n x m x 3. The
    range of the values is 0..255
    '''
    
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray/255.

In [361]:
def process_img(image, coords):
    image_cropped = image[coords[1]:coords[3], coords[0]:coords[2], :]
    image_gray  = rgb2gray(image_cropped)
    processed_image = imresize(image_gray, (32,32))
    
    return processed_image

In [362]:
def to_list(coord_string):
    return map(int,coord_string.split(","))


In [363]:
act = list(set([a.split("\n")[0] for a in open("subset_actors.txt").readlines()]))

In [364]:
#read line of image data
facescrub_acts = pd.read_table("all_acts.txt",sep="\t",header=None)
facescrub_acts.columns = ["Name", "Col1", "Col2", "URL", "Coordinates", "Col5"]

In [365]:
#Delete columns with seemingly gibberish data
facescrub_acts_trimmed = facescrub_acts.drop(facescrub_acts.columns[[1,2,5]], axis = 1)

In [382]:
facescrub_acts_trimmed

,Name,URL,Coordinates
0,Alec Baldwin,http://sarcastic-news.com/wp-content/uploads/2...,"463,450,1785,1772"
1,Alec Baldwin,http://images6.fanpop.com/image/photos/3280000...,"1111,1116,2869,2874"
2,Alec Baldwin,http://girlonthepark.files.wordpress.com/2011/...,"192,279,671,758"
3,Alec Baldwin,http://content6.flixster.com/rtactor/40/31/403...,"78,21,236,179"
4,Alec Baldwin,http://www.foxnews.com/images/278980/0_61_bald...,"113,20,268,175"
5,Alec Baldwin,http://www.vanityfair.com/hollywood/2012/07/al...,"112,176,383,447"
6,Alec Baldwin,http://blogs.villagevoice.com/dailymusto/image...,"141,108,316,283"
7,Alec Baldwin,http://www.msnbc.com/sites/msnbc/files/article...,"331,242,753,664"
8,Alec Baldwin,http://images.askmen.com/photos/alec-baldwin/8...,"48,56,254,262"
9,Alec Baldwin,http://upload.wikimedia.org/wikipedia/commons/...,"403,571,1104,1272"


In [366]:
#create directory for unprocessed images
current_directory = os.getcwd()
images = os.path.join(current_directory, r'images')
if not os.path.exists(images):
   os.makedirs(images)

In [ ]:
#create directory for each actor
for actor in act:
    actor_directory = os.path.join(images,actor)
    if not os.path.exists(actor_directory):
        os.makedirs(actor_directory)
        os.makedirs(os.path.join(actor_directory,actor+r'_unprocessed'))
        os.makedirs(os.path.join(actor_directory,actor+r'_processed'))


In [367]:
#iterate over the data frame SAMPLE to open the url and save image
i=0
for index, row in facescrub_acts_trimmed.iterrows():
    actor = row['Name']
    if actor in act:
          
        image_name = actor+'_{}.jpg'.format(i)
        unprocessed_dir = os.path.join(images,actor,actor+r'_unprocessed')
        processed_dir = os.path.join(images,actor, actor+r'_processed')
        image_name_w_path = os.path.join(unprocessed_dir,image_name)
        processed_image_w_path = os.path.join(processed_dir,image_name)
        
        if (row['URL'] == "http://www.sawfirst.com/wp-content/uploads/20"):
        
        else:
            try:
                #urllib.urlretrieve(row['URL'],image_name_w_path)
                timeout(urllib.urlretrieve, (row['URL'],image_name_w_path),{}, 2)
                image= imread(image_name_w_path) #check to see if image actually opens, if it doesn't handle exception
                image_processed = process_img(image, to_list(row['Coordinates'])) 
                imsave(processed_image_w_path, image_processed)

                #create data set
                if i == 0:
                    x = image_processed
                    y = np.array([actor])
                else: 
                    x = np.dstack((x, image_processed))
                    y = np.append(y,[actor])

                i+=1 #if image file doesn't open, i won't be incremented
                print(image_name_w_path)
                print("working:     "+row['URL'])

            except Exception:
                #if url is broken
                sys.exc_clear()
                print("not working:     "+row['URL'])


/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()


not working:     http://sarcastic-news.com/wp-content/uploads/2013/11/Alec_Baldwin_PETA_Shankbone_2008.jpg


/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.
/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_0.jpg
working:     http://images6.fanpop.com/image/photos/32800000/-Alec-Baldwin-alec-baldwin-32804837-4239-4265.jpg
not working:     http://girlonthepark.files.wordpress.com/2011/03/alec_baldwin03.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_1.jpg
working:     http://content6.flixster.com/rtactor/40/31/40316_pro.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_2.jpg
working:     http://www.foxnews.com/images/278980/0_61_baldwin_alec.jpg
not working:     http://www.vanityfair.com/hollywood/2012/07/alec-baldwin-leading-man-photo-gallery/_jcr_content/par/cn_contentwell/par-main/cn_slideshow/item0.rendition.slideshowWideVertical.alec-baldwin-ss01.jpg
not working:     http://blogs.villagevoice.com/daily

not working:     http://khojakhujee.com/wp-content/uploads/2013/11/Top-American-Actors-Alec-Baldwin.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_30.jpg
working:     http://images.smh.com.au/2012/06/20/3389562/alec_baldwin_729-420x0.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_31.jpg
working:     http://www.hollywoodreporter.com/sites/default/files/imagecache/blog_post_349_width/2013/05/alec_baldwin.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_32.jpg
working:     http://imstars.aufeminin.com/stars/fan/alec-baldwin/alec-baldwin-20061117-179839.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Alec Baldwin/Alec Baldwin_unprocessed/Alec Baldwin_33.jpg
working:     http://www.showbiz411.com/wp-content/uploads/20

...

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_836.jpg
working:     http://wfiles.brothersoft.com/a/angie-harmon-eyes_102243-1152x864.jpg
not working:     http://static.tvgcdn.net/MediaBin/Galleries/Editorial/130722/Red_Carpet/smallcrops/red-carpet-harmon1sm.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_837.jpg
working:     http://www.entertainmentwallpaper.com/images/desktops/celebrity/angie_harmon03.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_838.jpg
working:     http://www3.pictures.zimbio.com/gi/Angie%252BHarmon%252B7th%252BAnnual%252BKidstock%252BMusic%252BArt%252BIJSATtuQ_qhl.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_839.jpg
working:     http://i2.l

not working:     http://www.newsgab.com/attachments/celebrity-pictures/20163d1160792216-angie-harmon-gold-fragrance-collection-launch-black-dress-angie-harmon-gold-fragrance-collection-launch-08.jpg
not working:     http://www.hairstylescity.com/images/angie-harmon-3/Angie-Harmon-12875343.jpg
not working:     http://www.mashedreport.com/contributors/harmonmilk.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_865.jpg
working:     http://www.thewallpapersonline.com/wallpaper/angie-harmon/angie-harmon-34287.jpg
not working:     http://www.sawfirst.com/wp-content/uploads/2013/07/Angie-Harmon-1.jpg
not working:     http://media.celebrity-pictures.ca/Celebrities/Angie-Harmon/Angie-Harmon-1287544.jpg
not working:     http://live.drjays.com/wp-content/uploads/2011/07/angielawyer.jpg
not working:     http://www.fanphobia.net/uploads/actors/19280/Angie-Harmon-in-Renegade.jpg
not working:     http://cdn.blogs.sheknow

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_887.jpg
working:     http://4.bp.blogspot.com/-dS0Sdi3i2Xk/T-OjyFJV1tI/AAAAAAAABLE/nlbVtP5R1NI/s1600/Angie%252BHarmon%252BWallpapers%252B01.jpg
not working:     http://xbradtc.files.wordpress.com/2008/08/angie_harmon15.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_888.jpg
working:     http://ilarge.listal.com/image/238227/936full-angie-harmon.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_889.jpg
working:     http://www.hollywoodreporter.com/hr/photos/stylus/129219-angie_harmon_large.jpg
not working:     http://www.sawfirst.com/wp-content/uploads/2013/07/Angie-Harmon-9.jpg
not working:     http://cdn.blogs.sheknows.com/celebsalon.sheknows.com/2011/02/angie-harmon-hair1.jpg
not working:     http://i

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_910.jpg
working:     http://www.celebitchy.com/wp-content/uploads/2009/03/alo-062258.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_911.jpg
working:     http://basedefotos.com/files/images/2011/10/fotos-de-angie-harmon-8.jpg
not working:     http://www.sawfirst.com/wp-content/uploads/2013/07/Angie-Harmon-8.jpg
not working:     http://img005.lazygirls.info/people/selena_gomez/selena_gomez_selena_with_angie_harmon_TmUh8GwO.sized.png
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie Harmon_912.jpg
working:     http://lipstickpowdernpaint.com/wp-content/uploads/angie-harmon-at-the-2011-sag-awards.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Angie Harmon/Angie Harmon_unprocessed/Angie

/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/getpass.py:83: GetPassWarning: Can not control echo on the terminal.
  passwd = fallback_getpass(prompt, stream)
Enter password for df in Password Protected Area at www.angie-harmon.e-znane.pl: 

In [ ]:
# ##check 
# print(x.shape,y.shape)
# print(type(x), type(y))

In [408]:
#save dataset
np.save(os.path.join(os.getcwd(),"x.npy") ,x)
np.save(os.path.join(os.getcwd(),"y.npy") ,y)
